Python script to create a singlepoint energy calculation dataset
Usage: modify the setting lines below and run `python sp_dataset.py`

In [ ]:
import qcportal as ptl
from qcportal.molecules import Molecule
from qcportal.record_models import RecordStatusEnum
import numpy as np
import glob

Settings

In [ ]:
client = ptl.PortalClient("http://127.0.0.1:7777") # database server and port

In [ ]:
GEOM_FILE = 'test-5.xyz' # A stacked xyz file containing multiple water cluster geoms

In [ ]:
N = 3 # no. of water molecules in one cluster

In [ ]:
DELETE_SCRATCH = True # if false keeping the scratch files for debugging purpose

In [ ]:
DATASET_NAME = 'W3_singlepoint_test'

In [ ]:
DATASET_DESCRIPTION = 'Single point test calculations of water trimers'

In [ ]:
# specification
sp_spec = {
    "program": "psi4",
    "driver": "energy",
    "method": "b3lyp",
    "basis": "6-31G*",
    "keywords": {"e_convergence": 1e-10, "d_convergence": 1e-10},
}

In [ ]:
SPEC_NAME = 'b3lyp/6-31G*'

In [ ]:
ENTRY_PRE = 'trimer_'

End Settings

In [ ]:
i = -1

In [ ]:
row_length = 12*N # no. of elements for a geom row of a structure

In [ ]:
geom_blank = [" "]*row_length
geom = [geom_blank]

In [ ]:
j = 0

In [ ]:
with open(GEOM_FILE,'r') as Inp:
    for line in Inp:
        line_sp = line.split()
        if len(line_sp) == 1:
            i += 1
            j = 0
            if i != 0:
                geom = np.vstack([geom,geom_blank])
        elif len(line_sp) == 4:
            if i == 0:
                geom[j:j+4] = line_sp
            else:
                geom[i,j:j+4] = line_sp
            j += 4

In [ ]:
ds = client.add_dataset("singlepoint",
                        name=DATASET_NAME,
                        description=DATASET_DESCRIPTION)

In [ ]:
for ii in range(len(geom)):
    with open('tmp-geom'+str(ii)+'.in', 'w+') as GeomInp:
        for j in range(0,row_length,4):
            GeomInp.write("{} {} {} {}\n".format(geom[ii,j],float(geom[ii,j+1]),float(geom[ii,j+2]),float(geom[ii,j+3])))
            if ((j+4)%12 == 0 and ((j+4) < row_length)):
                GeomInp.write("--\n")
        GeomInp.seek(0)
        mol_geom = GeomInp.read()
        mol = Molecule.from_data(mol_geom)
    entry_name = ENTRY_PRE+str(ii)
    ds.add_entry(name=entry_name, molecule=mol)

In [ ]:
ds.add_specification(name=SPEC_NAME, specification=sp_spec)

In [ ]:
ds.submit()

In [ ]:
# clean scratch
if DELETE_SCRATCH:
    for f in glob.glob("tmp-geom*.in"):
        os.remove(f)

ds.print_status() # check the status of the submitted dataset

# get one record and print the single point energy
rec = ds.get_record('trimer_0', 'b3lyp/6-31G*')
print(rec.properties['return_energy'])